# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path


# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,-4.15,79,100,5.66,CA,1698325913
1,1,kismayo,-0.3582,42.5454,28.60,71,40,5.52,SO,1698325913
2,2,lata,40.1629,-8.3327,17.26,93,100,1.64,PT,1698325913
3,3,tsiombe,-25.3000,45.4833,25.82,42,17,6.15,MG,1698325913
4,4,grand falls-windsor,48.9332,-55.6649,9.72,95,99,3.66,CA,1698325913


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map
# Display the map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,lihue,21.9789,-159.3672,22.58,83,0,3.60,US,1698325914
25,25,wailua homesteads,22.0669,-159.3780,22.24,83,0,3.60,US,1698325918
261,261,loma de cabrera,19.4333,-71.6000,24.43,75,0,1.83,DO,1698325968
266,266,kapa'a,22.0752,-159.3190,22.99,83,0,3.60,US,1698325969
272,272,port elizabeth,-33.9180,25.5701,26.29,60,0,4.12,ZA,1698325970
276,276,san juan,-31.5375,-68.5364,21.10,51,0,0.95,AR,1698325971
313,313,nishinoomote,30.7333,131.0000,22.23,51,0,3.73,JP,1698325978
326,326,kailua-kona,19.6406,-155.9956,24.77,77,0,3.58,US,1698325764
340,340,holualoa,19.6228,-155.9522,22.70,74,0,3.58,US,1698325984
344,344,nageswari,25.9688,89.6930,25.25,72,0,2.28,BD,1698325985


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,lihue,US,21.9789,-159.3672,83,
25,wailua homesteads,US,22.0669,-159.3780,83,
261,loma de cabrera,DO,19.4333,-71.6000,75,
266,kapa'a,US,22.0752,-159.3190,83,
272,port elizabeth,ZA,-33.9180,25.5701,60,
276,san juan,AR,-31.5375,-68.5364,51,
313,nishinoomote,JP,30.7333,131.0000,51,
326,kailua-kona,US,19.6406,-155.9956,77,
340,holualoa,US,19.6228,-155.9522,74,
344,nageswari,BD,25.9688,89.6930,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lihue - nearest hotel: Kauai Palms
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
loma de cabrera - nearest hotel: Hotel El Maca
kapa'a - nearest hotel: Pono Kai Resort
port elizabeth - nearest hotel: Waterford Hotel
san juan - nearest hotel: Hotel San Francisco
nishinoomote - nearest hotel: ゼウスハウス
kailua-kona - nearest hotel: Kona Seaside Hotel
holualoa - nearest hotel: Kona Hotel
nageswari - nearest hotel: স্মৃতি রেস্ট হাউজ
ateli mandi - nearest hotel: No hotel found
bijawar - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
dinga - nearest hotel: No hotel found
chittagong - nearest hotel: Samiha Community Center and Convention Hall, Baraiyarhat, Chittagong


,City,Country,Lat,Lng,Humidity,Hotel Name
6,lihue,US,21.9789,-159.3672,83,Kauai Palms
25,wailua homesteads,US,22.0669,-159.3780,83,Hilton Garden Inn Kauai Wailua Bay
261,loma de cabrera,DO,19.4333,-71.6000,75,Hotel El Maca
266,kapa'a,US,22.0752,-159.3190,83,Pono Kai Resort
272,port elizabeth,ZA,-33.9180,25.5701,60,Waterford Hotel
276,san juan,AR,-31.5375,-68.5364,51,Hotel San Francisco
313,nishinoomote,JP,30.7333,131.0000,51,ゼウスハウス
326,kailua-kona,US,19.6406,-155.9956,77,Kona Seaside Hotel
340,holualoa,US,19.6228,-155.9522,74,Kona Hotel
344,nageswari,BD,25.9688,89.6930,72,স্মৃতি রেস্ট হাউজ


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)